In [6]:
import os
from os.path import join as p_join
import sys
sys.path.insert(0, '/'.join(os.getcwd().split('/')[:-1]))
import time
from tqdm.notebook import tqdm
import itertools
import collections
from datetime import datetime
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd
import scipy
from matplotlib_venn import *
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
import requests
pd.set_option('max.columns', 100)

In [9]:
DATE = '2020-01-01'
PROJECT_PATH = '..'

In [10]:
from src.parse_utils import (
    parse_supplies,
    parse_purchases,
    parse_sales,
    parse_storage,
    parse_report
)
from src.utils import create_wb_db_connection
from src.aggregate import make_daily_sales, make_unq_subjects_all_dates_df
eng = create_wb_db_connection()

Инициализируем подключение к базе...
ок


In [11]:
matplotlib.rcParams['figure.figsize'] = (8, 8)
sns.set_style('whitegrid')

---

In [12]:
DATE = '2021-01-01'

In [40]:
wb_key = open(p_join(PROJECT_PATH, 'configs', 'wildberries_api64.txt'), mode='r', encoding='utf-8').read()

In [41]:
# Поставки
supplies_df = parse_supplies(
    date_from=DATE,
    wb_key=wb_key,
#     var_name_dict=var_name_dict
)
supplies_df

,incomeId,number,date,lastChangeDate,supplierArticle,techSize,barcode,quantity,totalPrice,dateClose,warehouseName,nmId,status
0,2267245,,2021-03-22,2021-03-24 11:00:22.460,КН/001_розовый,0,2001254432003,11,0,2021-03-24,Подольск2,19463069,Принято
1,2267245,,2021-03-22,2021-03-24 11:00:22.460,КН/002_мятный,0,2001254428006,18,0,2021-03-24,Подольск2,19463070,Принято
2,2267245,,2021-03-22,2021-03-24 11:00:22.460,КН/003_серый,0,2001254441005,15,0,2021-03-24,Подольск2,19463071,Принято
3,2267245,,2021-03-22,2021-03-24 11:00:22.460,НСП/004_золотой_глянцевый,0,2001256726001,25,0,2021-03-24,Подольск2,19464860,Принято
4,2267245,,2021-03-22,2021-03-24 11:00:22.460,НСП/005_розовое_золото_глянцевый,0,2001256873002,10,0,2021-03-24,Подольск2,19464861,Принято
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,6363484,,2022-02-06,2022-02-06 16:00:05.687,НСП/008_золотой_матовый,0,2001256898005,1,0,2022-02-06,Подольск2,19464864,Принято
115,6363484,,2022-02-06,2022-02-06 16:00:05.687,mskmsk,0,2002387186009,15,0,2022-02-06,Подольск2,26999508,Принято
116,6363484,,2022-02-06,2022-02-06 16:00:05.687,teasp_вет/золотой,0,2002723742005,3,0,2022-02-06,Подольск2,28204982,Принято
117,6363484,,2022-02-06,2022-02-06 16:00:05.687,teasp_вет/серебро,0,2002727219008,5,0,2022-02-06,Подольск2,28222145,Принято


In [33]:
# Склад
storage_df = parse_storage(
    date_from=DATE,
    wb_key=wb_key,
#     var_name_dict=var_name_dict
)
storage_df

,lastChangeDate,supplierArticle,techSize,barcode,quantity,isSupply,isRealization,quantityFull,quantityNotInOrders,warehouseName,inWayToClient,inWayFromClient,nmId,subject,category,daysOnSite,brand,SCCode,Price,Discount
0,2021-12-31 02:49:31.163,teasp_вет/розовое_золото,0,2002727191007,1,True,False,1,0,Санкт-Петербург,0,0,28222072,Ложки чайные,Посуда и инвентарь,225,Auplum,,1240,50
1,2021-12-31 02:49:31.163,bandage1,L,2002852068014,0,True,False,1,0,Казань,1,0,28678954,Бандажи коленные,Здоровье,225,syho,,2264,50
2,2022-02-05 06:15:04.560,НСП/007_серебряный_глянцевый,0,2001256892003,0,True,False,0,0,Екатеринбург,0,0,19464863,Наборы столовых приборов,Посуда и инвентарь,350,Auplum,,3180,50
3,2022-02-05 06:15:04.560,mskmsk,0,2002387186009,0,True,False,0,0,Казань,0,0,26999508,Маски для сна,Аксессуары,277,syho,,780,50
4,2022-02-06 06:05:39.330,mskmsk,0,2002387186009,0,True,False,0,0,Екатеринбург,0,0,26999508,Маски для сна,Аксессуары,278,syho,,780,50
5,2022-02-07 05:40:07.283,НСП/004_золотой_глянцевый,0,2001256726001,8,True,False,8,8,Коледино,0,0,19464860,Наборы столовых приборов,Посуда и инвентарь,353,Auplum,,3180,50
6,2022-02-07 05:40:07.283,НСП/004_золотой_глянцевый,0,2001256726001,10,True,False,10,10,Екатеринбург,0,0,19464860,Наборы столовых приборов,Посуда и инвентарь,353,Auplum,,3180,50
7,2022-02-07 05:40:07.283,НСП/004_золотой_глянцевый,0,2001256726001,2,True,False,2,2,Санкт-Петербург,0,0,19464860,Наборы столовых приборов,Посуда и инвентарь,353,Auplum,,3180,50
8,2022-02-07 05:40:07.283,НСП/004_золотой_глянцевый,0,2001256726001,25,True,False,25,25,Электросталь,0,0,19464860,Наборы столовых приборов,Посуда и инвентарь,353,Auplum,,3180,50
9,2022-02-07 05:40:07.283,НСП/005_розовое_золото_глянцевый,0,2001256873002,0,True,False,7,7,Коледино,7,0,19464861,Наборы столовых приборов,Посуда и инвентарь,353,Auplum,,3180,50


In [59]:
# Заказы
purchase_df = parse_purchases(
    date_from=DATE,
    wb_key=wb_key,
    flag=0,
#     var_name_dict=var_name_dict
)
purchase_df

,number,date,lastChangeDate,supplierArticle,techSize,barcode,quantity,totalPrice,discountPercent,warehouseName,oblast,incomeID,odid,nmId,subject,category,brand,isCancel,cancel_dt,gNumber,sticker
0,30694528479,2021-08-31 22:16:12,2021-09-01 03:12:42,НСП/011_серебряный_матовый,0,2001256932006,1,1717.14,30,Екатеринбург,Тюменская,3537166,100653555980,19464867,Наборы столовых приборов,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,91195127552123899025,
1,30719157343,2021-09-01 10:25:35,2021-09-01 19:06:18,mskmsk,0,2002387186009,1,864.29,30,МЛП-Подольск,Крым,3247649,100654220523,26999508,Маски для сна,Аксессуары,syho,False,0001-01-01T00:00:00,8797302483076431065,
2,30769691007,2021-09-01 21:56:11,2021-09-01 22:02:15,НСП/009_розовое_золото_матовый,0,2001256903006,1,1844.29,30,Коледино,Московская,2781185,50670993681,19464865,Наборы столовых приборов,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,2288733453426571033,
3,30769691007,2021-09-01 21:56:11,2021-09-01 22:02:15,НСП/009_розовое_золото_матовый,0,2001256903006,1,1844.29,30,Электросталь,Московская,3537166,50670993685,19464865,Наборы столовых приборов,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,2288733453426571033,
4,30604107647,2021-08-30 20:10:53,2021-09-01 23:33:36,mskmsk,0,2002387186009,1,864.29,30,Коледино,Москва,3247649,100651029425,26999508,Маски для сна,Аксессуары,syho,True,2021-09-01T00:00:00,8967142728571077253,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2324,46085730463,2022-02-10 00:01:09,2022-02-10 06:03:41,НСП/007_серебряный_глянцевый,0,2001256892003,1,3180.00,63,Краснодар 2,Москва,4804244,201838156912,19464863,Наборы столовых приборов,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,93948253419281515143,
2325,46085730463,2022-02-10 00:01:09,2022-02-10 06:03:41,НСП/007_серебряный_глянцевый,0,2001256892003,1,3180.00,63,Краснодар 2,Москва,4804244,201838156913,19464863,Наборы столовых приборов,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,93948253419281515143,
2326,46085730463,2022-02-10 00:01:09,2022-02-10 06:34:02,НСП/007_серебряный_глянцевый,0,2001256892003,1,3180.00,63,Казань,Москва,3964378,201838156909,19464863,Наборы столовых приборов,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,93948253419281515143,
2327,46085730463,2022-02-10 00:01:09,2022-02-10 06:42:04,НСП/007_серебряный_глянцевый,0,2001256892003,1,3180.00,63,Казань,Москва,3964378,201838156910,19464863,Наборы столовых приборов,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,93948253419281515143,


In [49]:
unq_keys_df = pd.read_sql(
    """
    select 
        distinct supplierArticle
        , techSize
        , barcode
    from
        wb_yarik.sales
        
    union 
    
    select 
        distinct supplierArticle
        , techSize
        , barcode
    from
        wb_yarik.purchases
        
    union
    
    select 
        distinct supplierArticle
        , techSize
        , barcode
    from
        wb_yarik.storage
        
    union 
    
    select 
        distinct supplierArticle
        , techSize
        , barcode
    from
        wb_yarik.supplies
    """,
    eng
)
unq_keys_df['day'] = None
unq_keys_df['day'].loc[0] = pd.date_range(start='2021-01-01', end=str(datetime.now().date()))
unq_keys_df['day'].fillna(method='ffill', inplace=True)
unq_keys_df = unq_keys_df.explode('day')

In [48]:
from src.aggregate import make_daily_sales

Инициализируем подключение к базе...
ок


In [49]:
daily_sales_df = make_daily_sales()

In [50]:
daily_sales_df.shape

(1653, 33)

In [53]:
# Продажи
sales_df = parse_sales(
    date_from=DATE,
    wb_key=wb_key,
    flag=0,
#     var_name_dict=var_name_dict
)
sales_df

,number,date,lastChangeDate,supplierArticle,techSize,barcode,quantity,totalPrice,discountPercent,isSupply,isRealization,orderId,promoCodeDiscount,warehouseName,countryName,oblastOkrugName,regionName,incomeID,saleID,odid,spp,forPay,finishedPrice,priceWithDisc,nmId,subject,category,brand,IsStorno,gNumber,sticker
0,,2021-09-01 10:19:45,2021-09-01 10:30:51,НСП/004_золотой_глянцевый,0,2001256726001,1,2900,30,False,True,3.054720e+10,25,Новосибирск,Россия,Сибирский федеральный округ,Новосибирская,3537166,S1741183025,100649433498,0,1385.47,1386.47,1522.50,19464860,Наборы столовых приборов,Посуда и инвентарь,Auplum,0,91042353372731518546,
1,,2021-09-01 14:06:51,2021-09-01 14:14:54,КН/002_мятный,0,2001254428006,1,4104,30,False,True,3.055047e+10,25,Коледино,Россия,Центральный федеральный округ,Москва,2267245,S1742126327,100649515477,0,1960.69,1961.69,2154.60,19463070,Наборы кухонных принадлежностей,Посуда и инвентарь,Auplum,0,9859120058773614172,
2,1272674075,2021-09-01 15:45:31,2021-09-02 02:23:36,КН/002_мятный,0,2001254428006,-1,-4104,30,False,True,3.055047e+10,25,Коледино,Россия,Центральный федеральный округ,Москва,2267245,R68974662,100649515477,0,-1960.69,-1961.69,-2154.60,19463070,Наборы кухонных принадлежностей,Посуда и инвентарь,Auplum,0,9859120058773614172,
3,,2021-09-02 11:29:10,2021-09-02 11:33:36,НСП/011_серебряный_матовый,0,2001256932006,1,2695,30,False,True,3.069453e+10,25,Екатеринбург,Россия,Уральский федеральный округ,Тюменская,3537166,S1744466980,100653555980,0,1287.54,1288.54,1414.88,19464867,Наборы столовых приборов,Посуда и инвентарь,Auplum,0,91195127552123899025,
4,,2021-09-02 14:44:08,2021-09-02 14:49:39,mskmsk,0,2002387186009,1,1332,30,False,True,3.065170e+10,25,Санкт-Петербург,Россия,Северо-западный федеральный округ,Санкт-Петербург,3247649,S1745288671,100652352563,0,594.40,605.00,699.30,26999508,Маски для сна,Аксессуары,syho,0,96786532666405045542,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,,2022-02-07 17:07:44,2022-02-07 17:08:12,mskmsk,0,2002387186009,1,1040,50,False,True,4.493939e+10,25,Коледино,Россия,Уральский федеральный округ,Свердловская,5560043,S2348137119,151135792935,0,501.71,502.71,590.25,26999508,Маски для сна,Аксессуары,syho,0,9405846102809267370,
1871,,2022-02-07 17:07:44,2022-02-07 17:08:12,mskmsk,0,2002387186009,1,1040,50,False,True,4.493939e+10,25,Казань,Россия,Уральский федеральный округ,Свердловская,5560043,S2348139476,151135792934,0,501.71,502.71,590.25,26999508,Маски для сна,Аксессуары,syho,0,9405846102809267370,
1872,,2022-02-07 17:07:44,2022-02-07 17:08:12,mskmsk,0,2002387186009,1,1040,50,False,True,4.493939e+10,25,Коледино,Россия,Уральский федеральный округ,Свердловская,5560043,S2348144093,151135792936,0,501.71,502.71,590.25,26999508,Маски для сна,Аксессуары,syho,0,9405846102809267370,
1873,,2022-02-07 19:10:43,2022-02-07 19:10:47,НСП/007_серебряный_глянцевый,0,2001256892003,1,4240,50,False,True,4.528009e+10,0,Электросталь,,,,4804109,S2349127212,151652284053,0,1185.73,1186.73,1303.00,19464863,Наборы столовых приборов,Посуда и инвентарь,Auplum,0,93656155368199071578,


In [18]:
# Отчёт о продажах по реализации
report_df = parse_report(
    date_from=DATE,
    wb_key=wb_key,
    limit=100_000,
#     var_name_dict=var_name_dict
)
report_df

,realizationreport_id,suppliercontract_code,rrd_id,gi_id,subject_name,nm_id,brand_name,sa_name,ts_name,barcode,doc_type_name,quantity,retail_price,retail_amount,sale_percent,commission_percent,office_name,supplier_oper_name,order_dt,sale_dt,rr_dt,shk_id,retail_price_withdisc_rub,delivery_amount,return_amount,delivery_rub,gi_box_type_name,product_discount_for_report,supplier_promo,rid,ppvz_spp_prc,ppvz_kvw_prc_base,ppvz_kvw_prc,ppvz_sales_commission,ppvz_for_pay,ppvz_reward,ppvz_vw,ppvz_vw_nds,ppvz_office_id,ppvz_office_name,ppvz_supplier_id,ppvz_supplier_name,ppvz_inn,declaration_number,bonus_type_name
0,4646775,None,4355417958,3964378,Миски для животных,38122967,imilive,petmsk01black,0,2006953679001,Продажа,1,3174,1106.93,62.5,0.12,Коледино,Продажа,2021-09-29T00:00:00Z,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,3648548264,1190.25,0,0,0,Без коробов,50,25,50721791565,0.0583,0.1000,0.0417,49.59,1047.42,0.0,49.59,9.92,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,NaN,NaN
1,4646775,None,4355417961,3964378,Наборы столовых приборов,19464864,Auplum,НСП/008_золотой_матовый,0,2001256898005,Продажа,1,2638,1301.85,47.5,0.07,Коледино,Продажа,2021-09-28T00:00:00Z,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,3648376766,1384.95,0,0,0,Без коробов,30,25,50720302539,0.0500,0.0583,0.0083,11.54,1288.00,0.0,11.54,2.31,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,NaN,NaN
2,4646775,None,4355417962,3964378,Бандажи коленные,28678954,syho,bandage1,S,2002852046005,Продажа,0,0,0.00,0.0,0.00,Склад МЛП-Подольск 5,Логистика,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,3648368725,0.00,0,1,33,Моно,0,0,100719414262,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,NaN,NaN
3,4646775,None,4355417963,3964378,Органайзеры для хранения,38289400,Auplum,org01gray,0,2007029639004,Продажа,0,0,0.00,0.0,0.00,Коледино,Логистика,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,3648545313,0.00,1,0,105,Моно,0,0,100732857652,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,NaN,NaN
4,4646775,None,4355417964,3964378,Миски для животных,38122967,imilive,petmsk01black,0,2006953679001,Продажа,1,3174,1130.74,62.5,0.12,Коледино,Продажа,2021-09-29T00:00:00Z,2021-10-03T00:00:00Z,2021-10-03T00:00:00Z,3648548266,1190.25,0,0,0,Без коробов,50,25,100724134006,0.0417,0.1000,0.0583,69.43,1047.42,0.0,69.43,13.89,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3662,6567413,None,5254372911,5560043,Маски для сна,26999508,syho,mskmsk,0,2002387186009,Продажа,1,1040,390.00,62.5,0.15,Коледино,Продажа,2021-12-29T00:00:00Z,2021-12-31T00:00:00Z,2021-12-31T00:00:00Z,4597243779,390.00,0,0,0,Без коробов,50,25,200499123973,0.0000,0.1250,0.1250,48.75,331.50,0.0,48.75,9.75,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,,NaN
3663,6567413,None,5254372912,5560043,Маски для сна,26999508,syho,mskmsk,0,2002387186009,Продажа,0,0,0.00,0.0,0.00,Коледино,Логистика,2022-01-02T00:00:00Z,2022-01-02T00:00:00Z,2022-01-02T00:00:00Z,4597243811,0.00,1,0,55,Микс,0,0,200591735389,0.0000,0.0000,0.0000,0.00,0.00,0.0,0.00,0.00,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,,NaN
3664,6567413,None,5254372913,5560043,Маски для сна,26999508,syho,mskmsk,0,2002387186009,Продажа,0,0,0.00,0.0,0.00,Коледино,Логистика,2022-01-02T00:00:00Z,2022-01-02T00:00:00Z,2022-01-02T00:00:00Z,4597243803,0.00,1,0,55,Микс,0,0,200613915065,0.0000,0.0000,0.0000,0.00,0.00,0.0,0.00,0.00,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,,NaN
3665,6567413,None,5254372914,5560043,Маски для сна,26999508,syho,mskmsk,0,2002387186009,Продажа,0,0,0.00,0.0,0.00,Коледино,Логистика,2022-01-02T00:00:00Z,2022-01-02T00:00:00Z,2022-01-02T00:00:00Z,4597243807,0.00,1,0,55,Микс,0,0,200593435009,0.0000,0.0000,0.0000,0.00,0.00,0.0,0.00,0.00,0,Пункт самовывоза (ПВЗ),0,NaN,NaN,,NaN


In [53]:
(
    set(supplies_df.columns) 
    & set(storage_df.columns)
    & set(purchase_df.columns)
    & set(sales_df.columns)
#     & set(report_df.columns)
)

{'barcode',
 'lastChangeDate',
 'nmId',
 'quantity',
 'supplierArticle',
 'techSize',
 'warehouseName'}

In [16]:
import sqlalchemy as sa

In [17]:
sql_db_credentials = json.load(
    open(p_join(PROJECT_PATH, 'configs', 'sql_db_creadentials.json'), mode='r', encoding='utf-8')
)

In [20]:
db = 'yarik'
eng = sa.create_engine(
    f"mysql+pymysql://{sql_db_credentials[db]['login']}:{sql_db_credentials[db]['password']}@{sql_db_credentials[db]['host']}:{sql_db_credentials[db]['port']}"
)
eng.connect()

In [23]:
sales_df.to_sql(
    schema='wb_yarik',
    name='sales',
    con=eng,
    if_exists='replace'
)

In [24]:
purchase_df.to_sql(
    schema='wb_yarik',
    name='purchases',
    con=eng,
    if_exists='replace'
)

In [25]:
storage_df.to_sql(
    schema='wb_yarik',
    name='storage',
    con=eng,
    if_exists='replace'
)

In [26]:
supplies_df.to_sql(
    schema='wb_yarik',
    name='supplies',
    con=eng,
    if_exists='replace'
)

In [27]:
report_df.to_sql(
    schema='wb_yarik',
    name='report',
    con=eng,
    if_exists='replace'
)

In [62]:
daily_sales_df = pd.read_sql(
    """
    with t_agg as 
    (
        select 
            supplierArticle
            , techSize
            , subject
            , category
            , brand
            , cast(date as date) as day
            , sum(quantity) as sum_quantity
            , sum(quantity * totalPrice) as sum_totalPrice
            , sum(quantity * pricewithdisc) as sum_pricewithdisc
            , sum(quantity * forPay) as sum_forPay
            , sum(quantity * finishedPrice) as sum_finishedPrice
        from
            wb_yarik.sales
        where
            quantity > 0
        group by
            supplierArticle
            , techSize
            , subject
            , category
            , brand
            , cast(date as date)
        order by
            cast(date as date) asc
            , subject
            , techSize
            , date asc
    )
    
    select
        *
    from
        t_agg
    """,
    eng
)
daily_sales_df.to_sql(
    schema='wb_yarik',
    name='daily_sales',
    con=eng,
    if_exists='replace'
)

In [17]:
daily_sales_df = pd.read_sql(
    """
    with t_daily_sales as 
        (   
            select 
                supplierArticle
                , techSize
                , subject
                , category
                , brand
                , barcode
                , cast(date as date) as day
                , sum(quantity) as sum_quantity
                , sum(quantity * totalPrice) as sum_totalPrice
                , sum(quantity * pricewithdisc) as sum_pricewithdisc
                , sum(quantity * forPay) as sum_forPay
                , sum(quantity * finishedPrice) as sum_finishedPrice
            from
                wb_yarik.sales
            where
                quantity > 0
            group by
                supplierArticle
                , techSize
                , subject
                , barcode
                , category
                , brand
                , cast(date as date)
            order by
                cast(date as date) asc
                , subject
                , techSize
                , date asc
        )
        
    select * from t_daily_sales
    """,
    eng
)
daily_sales_df.head()

,supplierArticle,techSize,subject,category,brand,barcode,day,sum_quantity,sum_totalPrice,sum_pricewithdisc,sum_forPay,sum_finishedPrice
0,bandage1,XL,Бандажи коленные,Здоровье,syho,2002852035009,2021-12-23,1.0,2264.0,849.00,721.65,849.0
1,bag01pink,0,Сумки,Аксессуары,Noco.bag,2007034322007,2021-12-23,1.0,5357.0,1647.28,1400.19,1647.0
2,НСП/010_черный_матовый,0,Наборы столовых приборов,Посуда и инвентарь,Auplum,2001256914002,2021-12-24,1.0,3973.0,1281.29,1127.54,1281.0
3,НСП/004_золотой_глянцевый,0,Наборы столовых приборов,Посуда и инвентарь,Auplum,2001256726001,2021-12-24,1.0,4240.0,1272.00,1119.36,1272.0
4,org01pink,0,Органайзеры для хранения,Хранение вещей,Auplum,2007029886002,2021-12-24,2.0,5068.0,1900.50,1672.44,1900.0


In [18]:
daily_storage_df = pd.read_sql(
    """
    with t_daily_storage as 
        (   
            select 
                supplierArticle
                , techSize
                , barcode
                , cast(lastChangeDate as date) as day
                , sum(quantityFull) as sum_quantityFull
                , sum(inWayToClient) as sum_inWayToClient
                , sum(inWayFromClient) as sum_inWayFromClient
            from
                wb_yarik.storage
            group by
                supplierArticle
                , techSize
                , barcode
                , cast(lastChangeDate as date)
        )
        
    select * from t_daily_storage
    """,
    eng
)
daily_storage_df.head()

,supplierArticle,techSize,barcode,day,sum_quantityFull,sum_inWayToClient,sum_inWayFromClient
0,НСП/004_золотой_глянцевый,0,2001256726001,2022-01-09,57.0,4.0,0.0
1,НСП/005_розовое_золото_глянцевый,0,2001256873002,2022-01-09,17.0,1.0,2.0
2,НСП/006_черный_глянцевый,0,2001256882004,2022-01-09,76.0,10.0,0.0
3,НСП/007_серебряный_глянцевый,0,2001256892003,2022-01-09,47.0,9.0,2.0
4,НСП/009_розовое_золото_матовый,0,2001256903006,2022-01-09,4.0,1.0,0.0


In [11]:
daily_sales_df = make_daily_sales()
print(f"{daily_sales_df.shape[0]:,} x {daily_sales_df.shape[1]:,}")
(
    daily_sales_df
    .sort_values(by=['subject', 'supplierArticle', 'techSize', 'day'], ascending=True)
)

850 x 22


,supplierArticle,techSize,subject,category,brand,barcode,day,avg_totalPrice,avg_pricewithdisc,avg_forPay,avg_finishedPrice,sum_sales,sum_returned,sum_purchases,sum_q_totalPrice,sum_q_pricewithdisc,sum_q_forPay,sum_q_finishedPrice,sum_q_totalPrice_returned,sum_q_pricewithdisc_returned,sum_q_forPay_returned,sum_q_finishedPrice_returned
823,bandage1,,Бандажи коленные,Здоровье,syho,2002852068014,2022-01-17,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
48,bandage1,L,Бандажи коленные,Здоровье,syho,2002852068014,2021-09-12,1505.0,790.13,671.61,781.500000,4.0,0.0,0.0,6020.0,3160.52,2686.44,3126.00,0.0,0.0,0.0,0.0
57,bandage1,L,Бандажи коленные,Здоровье,syho,2002852068014,2021-09-13,1505.0,790.13,671.61,789.000000,2.0,0.0,0.0,3010.0,1580.26,1343.22,1578.00,0.0,0.0,0.0,0.0
97,bandage1,L,Бандажи коленные,Здоровье,syho,2002852068014,2021-09-18,1505.0,790.13,711.12,719.000000,1.0,0.0,3.0,1505.0,790.13,711.12,719.00,0.0,0.0,0.0,0.0
103,bandage1,L,Бандажи коленные,Здоровье,syho,2002852068014,2021-09-19,1505.0,790.13,697.95,723.666667,3.0,0.0,1.0,4515.0,2370.39,2093.85,2171.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,bag01pink,0,Сумки,Аксессуары,Noco.bag,2007034322007,2021-12-20,5357.0,1647.28,1400.19,1543.000000,2.0,0.0,2.0,10714.0,3294.56,2800.38,3086.00,0.0,0.0,0.0,0.0
741,bag01pink,0,Сумки,Аксессуары,Noco.bag,2007034322007,2021-12-21,5357.0,1647.28,1400.19,1647.000000,1.0,0.0,1.0,5357.0,1647.28,1400.19,1647.00,0.0,0.0,0.0,0.0
748,bag01pink,0,Сумки,Аксессуары,Noco.bag,2007034322007,2021-12-23,5357.0,1647.28,1400.19,1647.000000,1.0,0.0,0.0,5357.0,1647.28,1400.19,1647.00,0.0,0.0,0.0,0.0
752,bag01pink,0,Сумки,Аксессуары,Noco.bag,2007034322007,2021-12-24,5357.0,1828.08,1553.87,1827.940000,2.0,0.0,2.0,10714.0,3656.16,3107.74,3655.88,0.0,0.0,0.0,0.0


In [86]:
sales_df[['supplierArticle', 'subject']].drop_duplicates().sort_values(by=['subject'])

,supplierArticle,subject
25,bandage1,Бандажи коленные
129,petklt01black,Колтунорезки
127,krz01gold,Корзинки
12,teasp_вет/черный,Ложки чайные
18,teasp_вет/серебро,Ложки чайные
49,teasp_вет/золотой,Ложки чайные
52,teasp_вет/розовое_золото,Ложки чайные
4,mskmsk,Маски для сна
27,petmsk01black,Миски для животных
1359,КН/003_серый,Наборы кухонных принадлежностей


In [84]:
sales_df.subject.unique()

array(['Наборы столовых приборов', 'Наборы кухонных принадлежностей',
       'Маски для сна', 'Ложки чайные', 'Бандажи коленные',
       'Миски для животных', 'Органайзеры для хранения', 'Сумки',
       'Корзинки', 'Колтунорезки'], dtype=object)

In [115]:
tmp = pd.read_sql(
    """
    with t_bandage as 
        (
            select  
                day
                , sum(sum_sales) as bandage_s
                -- , sum(sum_purchases) as bandage_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Бандажи коленные'
            group by
                day
            order by
                day asc
        ),

        t_koltunorezki as 
        (
            select  
                day
                , sum(sum_sales) as koltunorezki_s
                -- , sum(sum_purchases) as koltunorezki_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Колтунорезки'
            group by
                day
            order by
                day asc
        ),
        
        t_korzinki as 
        (
            select  
                day
                , sum(sum_sales) as korzinki_s
                -- , sum(sum_purchases) as korzinki_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Корзинки'
            group by
                day
            order by
                day asc
        ),
        
        t_spoons as 
        (
            select  
                day
                , sum(sum_sales) as spoons_s
                -- , sum(sum_purchases) as spoons_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Ложки чайные'
            group by
                day
            order by
                day asc
        ),
        
        t_kitchen as 
        (
            select  
                day
                , sum(sum_sales) as kitchen_s
                -- , sum(sum_purchases) as kitchen_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Наборы кухонных принадлежностей'
            group by
                day
            order by
                day asc
        ),
        
        t_dream_mask as 
        (
            select  
                day
                , sum(sum_sales) as dream_mask_s
                -- , sum(sum_purchases) as dream_mask_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Маски для сна'
            group by
                day
            order by
                day asc
        ),
        
        t_animal_plate as 
        (
            select  
                day
                , sum(sum_sales) as animal_plate_s
                -- , sum(sum_purchases) as animal_plate_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Миски для животных'
            group by
                day
            order by
                day asc
        ),
        
        t_cutlery as 
        (
            select  
                day
                , sum(sum_sales) as cutlery_s
                -- , sum(sum_purchases) as cutlery_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Наборы столовых приборов'
            group by
                day
            order by
                day asc
        ),
        
        t_organizer as 
        (
            select  
                day
                , sum(sum_sales) as organizer_s
                -- , sum(sum_purchases) as organizer_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Органайзеры для хранения'
            group by
                day
            order by
                day asc
        ),
        
        t_bags as 
        (
            select  
                day
                , sum(sum_sales) as bags_s
                -- , sum(sum_purchases) as bags_p
            from
                wb_yarik.daily_sales
            where
                subject = 'Сумки'
            group by
                day
            order by
                day asc
        )


    select
        case when
            t1.day is null then t2.day else t1.day end as day
        , case when
            t1.bandage_s is null then 0 else t1.bandage_s end as bandage_s
        , case when
            t2.koltunorezki_s is null then 0 else t2.koltunorezki_s end as koltunorezki_s
        , case when
            t3.korzinki_s is null then 0 else t3.korzinki_s end as korzinki_s
        , case when
            t4.spoons_s is null then 0 else t4.spoons_s end as spoons_s
        , case when
            t5.kitchen_s is null then 0 else t5.kitchen_s end as kitchen_s
        , case when
            t6.dream_mask_s is null then 0 else t6.dream_mask_s end as dream_mask_s
        , case when
            t7.animal_plate_s is null then 0 else t7.animal_plate_s end as animal_plate_s
        , case when
            t8.cutlery_s is null then 0 else t8.cutlery_s end as cutlery_s
        , case when
            t9.organizer_s is null then 0 else t9.organizer_s end as organizer_s
        , case when
            t10.bags_s is null then 0 else t10.bags_s end as bags_s
        
    from
        t_bandage t1
        left join
            t_koltunorezki t2
        on  
            t1.day = t2.day
            
        left join 
            t_korzinki t3
        on
            t1.day = t3.day
            
        left join 
            t_spoons t4
        on
            t1.day = t4.day
            
        left join 
            t_kitchen t5
        on
            t1.day = t5.day
            
        left join 
            t_dream_mask t6
        on
            t1.day = t6.day
            
        left join 
            t_animal_plate t7
        on
            t1.day = t7.day
            
        left join 
            t_cutlery t8
        on
            t1.day = t8.day
            
        left join 
            t_organizer t9
        on
            t1.day = t9.day
            
        left join 
            t_bags t10
        on
            t1.day = t10.day
    """,
    eng
)
tmp

,day,bandage_s,koltunorezki_s,korzinki_s,spoons_s,kitchen_s,dream_mask_s,animal_plate_s,cutlery_s,organizer_s,bags_s
0,2021-09-13,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2021-09-15,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0
2,2021-09-18,1.0,0.0,0.0,0.0,0.0,2.0,2.0,4.0,0.0,0.0
3,2021-09-19,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-09-20,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
5,2021-09-21,1.0,0.0,1.0,0.0,1.0,0.0,0.0,10.0,0.0,0.0
6,2021-09-27,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,2.0
7,2021-10-05,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
8,2021-10-13,2.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,4.0,2.0
9,2021-10-19,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0


In [124]:
import plotly.express as px

In [161]:
import plotly.graph_objects as go

plt.figure(figsize=(16, 6))
colors = ['blue', 'red', 'green', 'orange', 'yellow', 'pink', 'purple', 'brown', 'black']
colors = colors + [color for color in list(matplotlib.colors.cnames.keys()) if color not in colors]
fig = go.Figure()
for i, col in enumerate(tmp.columns[1:]):
    fig.add_trace(go.Scatter(x=tmp['day'], y=tmp[col],
                    mode='lines+markers', name=col))
fig.write_html("/Users/antonandreytsev/Desktop/plotly.html")
fig.show();

<Figure size 1152x432 with 0 Axes>